In [67]:
#################################### imports #####################################

import os
import pandas as pd
import numpy as np
import env
from sklearn.model_selection import train_test_split

############################# acquire main function ##############################


def get_connection(db, user=env.user, host=env.host, password=env.password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

def acquire_zillow_data():
    '''
    wrangle_zillow_data will acuire the telco_churn data and 
    proceed to drop redundant columns and non-usefull info in 
    addition to encoding categorical variables
    '''
    filename = "zillow_prepared.csv"

    if os.path.isfile(filename):
        return pd.read_csv(filename)
    else:
        # read the SQL query into a dataframe
        query = '''
        select bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, fips
        from properties_2017
        where propertylandusetypeid like '261'
        '''
        df = pd.read_sql(query, get_connection('zillow'))
        
        # saving to csv
        df.to_csv('zillow_prepared.csv', index = False)

        return pd.read_csv(filename)
    
################################ Prep Data #####################################     


def clean_zillow_data(df):
    '''
    clean_zillow_data will take in a single pandas dataframe, 
    with the expected formatting observed in zillow df.
    It will drop nulls,
    It will ,
    And it will 
    
    Arguments: df. a pandas dataframe
    return: df, a pandas dataframe (cleaned)
    '''
    #drop nulls
    df = df.dropna()
    #convert to int: everything except taxammount 
    df['bedroomcnt'] = df['bedroomcnt'].astype(int)
    df['bathroomcnt'] = df['bathroomcnt'].astype(int)
    df['calculatedfinishedsquarefeet'] = df['calculatedfinishedsquarefeet'].astype(int)
    df['yearbuilt'] = df['yearbuilt'].astype(int)
    df['fips'] = df['fips'].astype(int)
    df['taxvaluedollarcnt'] = df['taxvaluedollarcnt'].astype(int)
    
    #rename columns for easier reading
    df.rename(columns={"bedroomcnt": "bed_count", "bathroomcnt": "bath_count", 
                       "calculatedfinishedsquarefeet": "sqr_ft", "taxvaluedollarcnt": "tax_val_usd"})
    #reset index
    df.reset_index(drop=True)
    return df

################################ Split Data #####################################                         
                         
def split_zillow(df):
    '''
    split_zillow will take in a single pandas df referencing a cleaned
    version of zillow data, and will then split the data into train,
    validate, and test sets
    
    Arguments: df. a pandas dataframe
    return: train, validate, test: the pandas df split from orginal df 
    '''
    train_val, test = train_test_split(df, random_state = 1349, train_size = 0.8)
    train, validate = train_test_split(train_val, random_state = 1349, train_size = 0.7)
    return train, validate, test

################################ full wrangle Data ##################################### 

def wrangle_zillow():
    '''
    wrangle_zillow will go through the process of acquiring zillow data from 
    a local .csv if present, if not, aquire through a sql query, save the data to a local .csv
    then proceed with cleaning the data, then splitting into train, test, and validate
    '''
    return split_zillow(
        clean_zillow_data(
            acquire_zillow_data()))